In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob

In [2]:

# Define chessboard patterns
patterns = [
    {"pattern_size": (11, 7), "square_size": 0.010},   # C
    {"pattern_size": (7, 5),  "square_size": 0.010},   # A
    {"pattern_size": (5, 7),  "square_size": 0.010},   # B
    {"pattern_size": (5, 15), "square_size": 0.010},   # D
]

# Termination criteria for cornerSubPix
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

path_02 = "/Users/mathildevangkilde/Desktop/34759_final_project_raw/calib/image_02/data/*.png"
path_03 = "/Users/mathildevangkilde/Desktop/34759_final_project_raw/calib/image_03/data/*.png"

images_02 = sorted(glob.glob(path_02))
images_03 = sorted(glob.glob(path_03))

print("02 images:", len(images_02))
print("03 images:", len(images_03))

objpoints = []               # 3D real world positions 
imgpoints_image_02 = []      # 2D detected points in images 
imgpoints_image_03 = []      # 2D detected points in images 

for file_02, file_03 in zip(images_02, images_03):
    img02 = cv2.imread(file_02)
    img03 = cv2.imread(file_03)

    gray02 = cv2.cvtColor(img02, cv2.COLOR_BGR2GRAY)
    gray03 = cv2.cvtColor(img03, cv2.COLOR_BGR2GRAY)

    found_pattern = False
    
    for pat in patterns:
        pattern_size = pat["pattern_size"]
        square_size  = pat["square_size"]

        ret02, corners02 = cv2.findChessboardCorners(gray02, pattern_size, None)
        ret03, corners03 = cv2.findChessboardCorners(gray03, pattern_size, None)

        if ret02 and ret03:
            # Refine corner locations
            corners02 = cv2.cornerSubPix(gray02, corners02, (11,11), (-1,-1), criteria)
            corners03 = cv2.cornerSubPix(gray03, corners03, (11,11), (-1,-1), criteria)

            # Generate object points
            objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
            objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
            objp *= square_size

            objpoints.append(objp)
            imgpoints_image_02.append(corners02)
            imgpoints_image_03.append(corners03)

            # Draw corners
            cv2.drawChessboardCorners(img02, pattern_size, corners02, ret02)
            cv2.drawChessboardCorners(img03, pattern_size, corners03, ret03)

            # Combine images side by side
            combined = np.hstack((img02, img03))
            cv2.imshow('Stereo Pair', combined)
            cv2.waitKey(500)  # Display for 500ms

            found_pattern = True
            break

    if not found_pattern:
        print(f"⚠ No chessboard found in pair {file_02} / {file_03}")

cv2.destroyAllWindows()


02 images: 19
03 images: 19


In [3]:
# camera calibration for begge cameraer 
print("\nCalibrating camera 02 ...")
ret02, mtx02, dist02, rvecs02, tvecs02 = cv2.calibrateCamera(
    objpoints,
    imgpoints_image_02,
    gray02.shape[::-1],
    None,
    None
)

print("Calibrating camera 03 ...")
ret03, mtx03, dist03, rvecs03, tvecs03 = cv2.calibrateCamera(
    objpoints,
    imgpoints_image_03,
    gray03.shape[::-1],
    None,
    None
)


Calibrating camera 02 ...
Calibrating camera 03 ...


In [4]:
#stereo calibration, finder camera extrinsics (rotation R og translation T)

image_size = (gray02.shape[1], gray02.shape[0])  # (width, height)
print("\nImage size:", image_size)

criteria_stereo = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS,
                   100,
                   1e-5)


ret, mtx02, dist02, mtx03, dist03, R, T, E, F = cv2.stereoCalibrate(
    objpoints,
    imgpoints_image_02,
    imgpoints_image_03,
    mtx02, dist02,
    mtx03, dist03,
    image_size,
    criteria=criteria_stereo,
    flags=cv2.CALIB_FIX_INTRINSIC  # har allerede intrinsics
)

print("\nStereo calibration resultater:")
print("Rotation:\n", R)
print("Translation:\n", T)


Image size: (1392, 512)

Stereo calibration resultater:
Rotation:
 [[ 0.89906385 -0.37258989 -0.22991513]
 [ 0.33765504  0.92436191 -0.17760666]
 [ 0.27869924  0.08204773  0.95686723]]
Translation:
 [[-1.35645537]
 [ 3.21200521]
 [-1.17976744]]


In [5]:
#stereo rectification 

R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
    cameraMatrix1=mtx02,
    distCoeffs1=dist02,
    cameraMatrix2=mtx03,
    distCoeffs2=dist03,
    imageSize=image_size,
    R=R,
    T=T,
    flags=cv2.CALIB_ZERO_DISPARITY,
    alpha=0  # 0 = no black borders, 1 = keep all image content
)

print("stereo Rectification resultater:")
print("Q matrix (for 3D reconstruction):\n", Q)

# Build undistort/rectify maps for fast remapping
map02_x, map02_y = cv2.initUndistortRectifyMap(
    mtx02, dist02, R1, P1, image_size, cv2.CV_32FC1
)

map03_x, map03_y = cv2.initUndistortRectifyMap(
    mtx03, dist03, R2, P2, image_size, cv2.CV_32FC1
)



stereo Rectification resultater:
Q matrix (for 3D reconstruction):
 [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  5.52992077e+01]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -1.29413735e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.12181089e+04]
 [ 0.00000000e+00  0.00000000e+00 -2.71674944e-01  0.00000000e+00]]


In [ ]:
#formattering af resultaterne fra rectification så vi kan sammenligne det 

def print_matrix(name, M):
    """Pretty print a matrix in one line, scientific notation."""
    M_flat = M.reshape(-1)
    print(f"{name}: " + " ".join([f"{v:.6e}" for v in M_flat]))

def print_vector(name, v):
    v_flat = v.reshape(-1)
    print(f"{name}: " + " ".join([f"{x:.6e}" for x in v_flat]))


# kamera 02
print("S_02:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("K_02", mtx02)
print_vector("D_02", dist02)
print_matrix("R_02", R)                # global stereo rotation applied to cam02
print_vector("T_02", T)

# rectified 02 
print("S_rect_02:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("R_rect_02", R1)
print_matrix("P_rect_02", P1)

print("\n")

# kamera 03
print("S_03:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("K_03", mtx03)
print_vector("D_03", dist03)
print_matrix("R_03", np.eye(3))        # cam03 reference rotation (identity)
print_vector("T_03", np.zeros((3,1)))  # reference camera at origin

# rectified 03
print("S_rect_03:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("R_rect_03", R2)
print_matrix("P_rect_03", P2)

# TODO tjek op på hvordan jeg får flere skakbrætter med 




=================== YOUR CALIBRATION (MATCHING PROVIDED FORMAT) ===================

S_02: 1.392000e+03 5.120000e+02
K_02: 1.998960e+03 0.000000e+00 7.526702e+02 0.000000e+00 1.428453e+03 2.314281e+02 0.000000e+00 0.000000e+00 1.000000e+00
D_02: 9.660750e+01 -3.116005e+03 5.408466e-01 -2.655764e+00 -8.686708e+03
R_02: 8.990638e-01 -3.725899e-01 -2.299151e-01 3.376550e-01 9.243619e-01 -1.776067e-01 2.786992e-01 8.204773e-02 9.568672e-01
T_02: -1.356455e+00 3.212005e+00 -1.179767e+00
S_rect_02: 1.392000e+03 5.120000e+02
R_rect_02: 9.811850e-01 5.924398e-02 -1.837557e-01 -1.260003e-01 9.176255e-01 -3.769448e-01 1.462872e-01 3.930059e-01 9.078251e-01
P_rect_02: 1.121811e+04 0.000000e+00 -5.529921e+01 0.000000e+00 0.000000e+00 1.121811e+04 1.294137e+03 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00


S_03: 1.392000e+03 5.120000e+02
K_03: 3.923678e+02 0.000000e+00 6.043924e+02 0.000000e+00 8.557116e+02 3.753683e+02 0.000000e+00 0.000000e+00 1.000000e+00
D_03: -5.576430e+0